In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor

In [2]:
#https://www.kaggle.com/datasets/ramprasad273/predicting-food-delivery-time
df = pd.read_excel("Participants Data/Data_Train.xlsx")

In [3]:
df = df.rename(columns={"Delivery_Time": "Preparation_Time"})

In [4]:
df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Preparation_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [5]:
df["Average_Cost"].unique()

array(['₹200', '₹100', '₹150', '₹250', '₹650', '₹350', '₹800', '₹50',
       '₹400', '₹600', '₹300', '₹750', '₹450', '₹550', '₹1,000', '₹500',
       '₹900', '₹1,200', '₹950', '₹850', '₹700', '₹1,150', 'for',
       '₹1,100', '₹1,400', '₹2,050'], dtype=object)

In [6]:
df.drop(df[df["Average_Cost"] == "for"].index, inplace=True)

In [7]:
df["Average_Cost"].value_counts()

₹200      3241
₹100      2557
₹150      2462
₹250       881
₹300       537
₹350       283
₹400       282
₹50        265
₹600       154
₹500       101
₹450        63
₹550        60
₹650        55
₹800        44
₹750        38
₹900        15
₹700        15
₹1,000      12
₹850        12
₹1,200       8
₹950         4
₹1,150       1
₹1,100       1
₹1,400       1
₹2,050       1
Name: Average_Cost, dtype: int64

In [8]:
df["Preparation_Time"] = df["Preparation_Time"].apply(lambda x: int(x.split()[0]))


In [9]:
df["Average_Cost"] = df["Average_Cost"].apply(lambda x: int(x[1:].replace(",","")))

In [10]:
df["Minimum_Order"] = df["Minimum_Order"].apply(lambda x: int(x[1:]))

In [11]:
df.shape

(11093, 9)

In [12]:
tmp = []
for i in range(df.shape[0]):
    if df["Rating"].iloc[i] == "-":
        continue
    elif df["Rating"].iloc[i] == "NEW":
        continue
    elif df["Rating"].iloc[i] == "Opening Soon":
        continue
    elif df["Rating"].iloc[i] == "Temporarily Closed":
        continue
    else:
        tmp.append(df["Rating"].iloc[i])

In [13]:
avg_rate = np.array(tmp).astype(float).mean()
avg_rate

3.6134596429744827

In [14]:
df["Rating"] = df["Rating"].apply(lambda x: avg_rate if x in ["NEW", "-", "Opening Soon", "Temporarily Closed"] else x).astype(float)


In [15]:
from hashlib import sha256

In [16]:
s = "FTI College, Law College Road, Pune"
int(sha256(s.encode()).hexdigest(), 16) % (10 ** 8)

16347948

In [17]:
def str2int(s):
    return int(sha256(s.encode()).hexdigest(), 16) % (10 ** 8)

In [18]:
df["Votes"] = df["Votes"].apply(lambda x: float(x) if x != "-" else 0)
df["Reviews"] = df["Reviews"].apply(lambda x: float(x) if x != "-" else 0)

In [19]:
df["Location"] = df["Location"].apply(str2int)

In [20]:
df["Cuisines"] = df["Cuisines"].apply(str2int)

In [21]:
df.columns

Index(['Restaurant', 'Location', 'Cuisines', 'Average_Cost', 'Minimum_Order',
       'Rating', 'Votes', 'Reviews', 'Preparation_Time'],
      dtype='object')

In [22]:
df

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Preparation_Time
0,ID_6321,16347948,68420766,200,50,3.50000,12.0,4.0,30
1,ID_2882,9753282,4745044,100,50,3.50000,11.0,4.0,30
2,ID_1595,87454491,25397617,150,50,3.60000,99.0,30.0,65
3,ID_5929,33210479,44047920,250,99,3.70000,176.0,95.0,30
4,ID_6123,64748422,67762926,200,99,3.20000,521.0,235.0,65
...,...,...,...,...,...,...,...,...,...
11089,ID_8067,39333219,15885595,250,50,4.20000,326.0,189.0,30
11090,ID_4620,37963703,46139923,100,50,3.60000,36.0,16.0,30
11091,ID_3392,10584479,55298936,100,50,3.50000,45.0,18.0,30
11092,ID_4115,9753282,59943232,100,50,3.10000,24.0,9.0,30


In [23]:
train_X = df[["Location", "Cuisines", "Average_Cost", "Minimum_Order", "Rating", "Votes", "Reviews"]].values

In [24]:
train_y = df["Preparation_Time"].values

In [25]:
reg = RandomForestRegressor(n_estimators=1000, max_depth=7, random_state=0)

In [26]:
reg.fit(train_X, train_y)

RandomForestRegressor(max_depth=7, n_estimators=1000, random_state=0)

In [27]:
reg.score(train_X, train_y)

0.403090677116975

In [28]:
import sklearn
from sklearn import ensemble

In [49]:
# Import Random forest architecture model from sklearn
from sklearn.ensemble import RandomForestRegressor
# Construct a random forest regressor
reg = RandomForestRegressor(n_estimators=1000, max_depth=7, random_state=0)
# Fit the model to the training data
reg.fit(train_X, train_y)
# score the model (R^2 score)
score = reg.score(train_X, train_y)
print("Score for Random Forest Regressor: ", score)

Score for Random Forest Regressor:  0.403090677116975


In [53]:
from sklearn.model_selection import cross_validate

In [54]:
cv_results = cross_validate(reg, train_X, train_y, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'), return_train_score=False)

In [55]:
pd.DataFrame(cv_results)

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_r2
0,8.715386,0.146983,-6.580655,-104.912154,0.252921
1,8.632098,0.144021,-6.599236,-103.172889,0.313661
2,8.730211,0.144274,-6.390216,-102.641398,0.312068
3,8.644535,0.145995,-6.743796,-109.902305,0.311915
4,8.596544,0.144996,-6.972394,-133.625194,0.235933


In [33]:
# Import Random forest architecture model from sklearn
from sklearn.ensemble import GradientBoostingRegressor
# Construct a random forest regressor
reg = GradientBoostingRegressor(n_estimators=1000, max_depth=7, random_state=0)
# Fit the model to the training data
reg.fit(train_X, train_y)
# score the model (R^2 score)
score = reg.score(train_X, train_y)
print("Score for GradientBoosting Regressor: ", score)


Score for GradientBoosting Regressor:  0.973051704897379


In [34]:
cv_results = cross_validate(reg, train_X, train_y, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'), return_train_score=False)

In [35]:
pd.DataFrame(cv_results)

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_r2
0,12.546615,0.043998,-5.746036,-92.276210,0.342901
1,12.444278,0.046001,-6.018684,-98.237235,0.346495
2,12.411858,0.045002,-5.582568,-90.936913,0.390515
3,12.516021,0.043003,-6.125243,-105.443386,0.339831
4,12.460207,0.045000,-6.166170,-117.594330,0.327597


In [36]:
pred = reg.predict(train_X)

In [37]:
pred

array([29.62543073, 29.37340423, 63.37239035, ..., 30.83259571,
       31.47702497, 30.30429838])

In [38]:
train_y

array([30, 30, 65, ..., 30, 30, 30], dtype=int64)

In [39]:
pd.DataFrame({"predict": pred, "actual": train_y}).sample(10)

,predict,actual
2051,30.024269,30
8833,30.841784,30
430,41.629505,45
9916,41.896415,45
3033,64.705605,65
287,30.198131,30
1998,30.030858,30
1690,30.418994,30
4307,31.029370,30
8227,30.385409,30


In [40]:
df_without_target = df.iloc[:,1:-1]
df_without_target.head()

,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
0,16347948,68420766,200,50,3.5,12.0,4.0
1,9753282,4745044,100,50,3.5,11.0,4.0
2,87454491,25397617,150,50,3.6,99.0,30.0
3,33210479,44047920,250,99,3.7,176.0,95.0
4,64748422,67762926,200,99,3.2,521.0,235.0


In [41]:
import time
import numpy as np

In [52]:
n = 100
durations = []
for i in range(n):
    start_time = time.time()
    sample_data = df_without_target.sample(10).values
    result = reg.predict(sample_data)
    end_time = time.time()
    duration = end_time - start_time
    durations.append(duration)
duration_mean = np.mean(durations)
duration_median = np.median(durations)
duration_std =  np.std(durations)
print("Mean duration: {:.2f} seconds".format(duration_mean))
print("Median duration: {:.2f} seconds".format(duration_median))
print("Standard deviation: {:.2f} seconds".format(duration_std))

Mean duration: 0.05 seconds
Median duration: 0.05 seconds
Standard deviation: 0.00 seconds


In [43]:
n = 100
durations = []
for i in range(n):
    start_time = time.time()
    sample_data = df_without_target.sample(1)
    result = reg.predict(sample_data)
    end_time = time.time()
    duration = end_time - start_time
    durations.append(duration)
duration_mean = np.mean(durations)
duration_median = np.median(durations)
duration_std =  np.std(durations)
print("Mean duration: {:.2f} seconds".format(duration_mean))
print("Median duration: {:.2f} seconds".format(duration_median))
print("Standard deviation: {:.2f} seconds".format(duration_std))

c:\Users\PoraRee\.conda\envs\EAT\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
c:\Users\PoraRee\.conda\envs\EAT\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
c:\Users\PoraRee\.conda\envs\EAT\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
c:\Users\PoraRee\.conda\envs\EAT\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
c:\Users\PoraRee\.conda\envs\EAT\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
c:\Users\PoraRee\.conda\envs\EAT\lib\site-packages\sklearn\base.py:443

Mean duration: 0.00 seconds
Median duration: 0.00 seconds
Standard deviation: 0.00 seconds


c:\Users\PoraRee\.conda\envs\EAT\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
c:\Users\PoraRee\.conda\envs\EAT\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
c:\Users\PoraRee\.conda\envs\EAT\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
c:\Users\PoraRee\.conda\envs\EAT\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [44]:
df.shape

(11093, 9)

In [45]:
# import geopandas
# gdf = geopandas.tools.geocode(df["Location"].iloc[:100])

In [46]:
# gdf["address"].iloc[0]